In [82]:
import pandas as pd
from pandas import Series,DataFrame
import numpy as np

In [83]:
url_front = 'http://data.insideairbnb.com/singapore/sg/singapore/'
url_end = '/data/listings.csv.gz'

# Clean Listings

In [84]:
dates = ['2019-03-18', '2019-04-20', '2019-05-23', '2019-06-25', '2019-07-21', '2019-08-28', '2019-09-25', '2019-10-25', '2019-11-26', '2019-12-28',
         '2020-01-26', '2020-02-27', '2020-03-21', '2020-04-26', '2020-05-27', '2020-06-22', '2020-10-26', '2020-11-27', '2020-12-29','2021-01-27']

In [85]:
listings = []

In [86]:
for date in dates:
    # print(date)
    listing = pd.read_csv(url_front+date+url_end)
    listings.append(listing)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (42,43,61,62,94,95) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [90]:
# 2020-10 onwards some features were lost.
long_features = listings[0].columns.values
short_features = listings[-1].columns.values
keep_features = [x for x in long_features if x in short_features]

In [91]:
keep_features

['id',
 'listing_url',
 'scrape_id',
 'last_scraped',
 'name',
 'description',
 'neighborhood_overview',
 'picture_url',
 'host_id',
 'host_url',
 'host_name',
 'host_since',
 'host_location',
 'host_about',
 'host_response_time',
 'host_response_rate',
 'host_acceptance_rate',
 'host_is_superhost',
 'host_thumbnail_url',
 'host_picture_url',
 'host_neighbourhood',
 'host_listings_count',
 'host_total_listings_count',
 'host_verifications',
 'host_has_profile_pic',
 'host_identity_verified',
 'neighbourhood',
 'neighbourhood_cleansed',
 'neighbourhood_group_cleansed',
 'latitude',
 'longitude',
 'property_type',
 'room_type',
 'accommodates',
 'bathrooms',
 'bedrooms',
 'beds',
 'amenities',
 'price',
 'minimum_nights',
 'maximum_nights',
 'minimum_minimum_nights',
 'maximum_minimum_nights',
 'minimum_maximum_nights',
 'maximum_maximum_nights',
 'minimum_nights_avg_ntm',
 'maximum_nights_avg_ntm',
 'calendar_updated',
 'has_availability',
 'availability_30',
 'availability_60',
 'avail

In [93]:
listings_full = listings[0][keep_features]
for i in range(1, 19):
    listings_full = pd.concat([listings_full, listings[i][keep_features]])

In [94]:
delete_col = ['listing_url',
 'scrape_id',
 'last_scraped',
 'picture_url',
 'host_url',
 'host_name',
 'host_location',
 'host_acceptance_rate',
 'host_thumbnail_url',
 'host_picture_url',
 'host_listings_count',
 'host_total_listings_count',
 'neighbourhood',
 'has_availability',
 'calendar_last_scraped',
 'first_review',
 'last_review',
 'license',
 'calculated_host_listings_count_entire_homes',
 'calculated_host_listings_count_private_rooms',
 'calculated_host_listings_count_shared_rooms']

In [95]:
listings_cleaned = listings_full.drop(columns = delete_col)

In [96]:
listings_cleaned = listings_cleaned.drop_duplicates()
listings_cleaned = listings_cleaned.reset_index().drop(columns = ['index'])

In [97]:
listings_cleaned.shape

(138263, 51)